In [ ]:
import os
import io
import pyodbc
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from dotenv import load_dotenv

now = datetime.now()
dia = str(now.day).zfill(2)
mes = str(now.month).zfill(2)
ano = str(now.year)
data_atual = f"{dia}-{mes}-{ano}"

CAMINHO_ENVS = r"\\192.168.0.210\exata$\BUSINESS INTELLIGENCE\bucket-exata\config\chave-json"

# Carrega os .env específicos
load_dotenv(os.path.join(CAMINHO_ENVS, "sql.env"))
load_dotenv(os.path.join(CAMINHO_ENVS, "path.env"))
load_dotenv(os.path.join(CAMINHO_ENVS, "creds.env"))

# Agora você pode acessar as variáveis
CAMINHO_BASE = os.getenv("CAMINHO_BASE")
SERVICE_ACCOUNT_FILE = os.path.join(CAMINHO_BASE, os.getenv("SERVICE_ACCOUNT_SUBPATH"))
SERVER = os.getenv("SERVER")
INSTANCE = os.getenv("INSTANCE")
DRIVER = os.getenv("DRIVER")
DATABASE = "TESTE"
UID = os.getenv("UID")
PWD = os.getenv("PWD")

In [ ]:
connection_string = f'DRIVER={DRIVER};SERVER={SERVER}\\{INSTANCE};DATABASE={DATABASE};UID={UID};PWD={PWD}'

try:
    conn = pyodbc.connect(connection_string)
    print("Conectou com sucesso!")
except pyodbc.Error as e:
    print("Erro ao conectar ao banco de dados:", e)
    conn = None

if conn:
    query = "SELECT caminho_pasta, id_arquivo, nome_arquivo, tipo_arquivo FROM vw_caminho_arquivos"
    df = pd.read_sql(query, conn)
    conn.close()
    print(df.head())

In [ ]:
# Autenticação Google Drive
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=['https://www.googleapis.com/auth/drive.readonly']
)
drive_service = build('drive', 'v3', credentials=credentials)

In [ ]:
for index, row in df.iterrows():
    # Caminho completo do arquivo (sem criar pasta com o nome do arquivo)
    caminho_final = os.path.join(
        CAMINHO_BASE,
        "T.I",
        "n8n",
        "Backup",
        data_atual,
        row['caminho_pasta'],
        row['nome_arquivo']  # apenas o nome do arquivo, não cria pasta
    )
    
    # Cria todas as pastas do caminho, exceto o arquivo
    os.makedirs(os.path.dirname(caminho_final), exist_ok=True)
    
    # Baixa arquivo do Drive
    request = drive_service.files().get_media(fileId=row['id_arquivo'])
    with io.FileIO(caminho_final, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Baixando {row['nome_arquivo']}... {int(status.progress() * 100)}%")
    
    print(f"Arquivo salvo em: {caminho_final}\n")